## Import

In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import pandas_profiling

In [2]:
class CFG:
    SEED = 42

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [5]:
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [6]:
test

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,TEST_000,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,TEST_001,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,TEST_002,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,TEST_003,0,0,0,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,TEST_004,0,0,0,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,TEST_170,0,0,0,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,TEST_171,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,TEST_172,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,TEST_173,0,0,0,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [6]:
# pr_train = train.profile_report()
# pr_test = test.profile_report()

In [7]:
# pr_train.to_file('./pr_train_report.html')

# pr_test.to_file('./pr_test_report.html')



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\ppjjh\AppData\Roaming\Python\Python39\site-packages\multimethod\__init__.py:312: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\ppjjh\AppData\Roaming\Python\Python39\site-packages\multimethod\__init__.py:312: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [9]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [10]:
train_x


,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A
1,0,0,0,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A
2,0,0,0,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A
3,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G
4,0,0,0,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A
258,0,0,0,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A
259,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G
260,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G


## Data Pre-processing
### Label-Encoding

In [11]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [15]:
snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

In [16]:
snp_data

['G G',
 'A G',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'A G',
 'G G',
 'A A',
 'A A',
 'A G',
 'A A',
 'G G',
 'G G',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'A G',
 'G G',
 'A A',
 'G G',
 'A G',
 'A A',
 'A A',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'A A',
 'G G',
 'A A',
 'A A',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'A A',
 'A A',
 'G G',
 'A A',
 'G G',
 'A A',
 'A A',
 'A G',
 'A G',
 'A A',
 'G G',
 'A A',
 'G G',
 'G G',
 'A G',
 'A G',
 'A A',
 'A G',
 'G G',
 'A G',
 'G G',
 'G G',
 'G G',
 'A A',
 'A G',
 'G G',
 'A G',
 'A G',
 'G G',
 'A A',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'G G',
 'A G',
 'A A',
 'A G',
 'A G',
 'A G',
 'G G',
 'A G',
 'G G',
 'A A',
 'A G',
 'G G',
 'G G',
 'G G',
 'A G',
 'G G',
 'G G',
 'A G',
 'A G',
 'G G',
 'G G',
 'G G',
 'A A',
 'A G',
 'A G',
 'G G',
 'G G',
 'G G',
 'A G',
 'A A',


In [17]:
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [18]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

## Model Fit

In [35]:
clf = RandomForestClassifier(random_state=CFG.SEED,
                             n_estimators = 100,
                             criterion= "gini",
                             max_depth= 100,
                             min_samples_split= 2,
                             min_samples_leaf= 1,
                             min_weight_fraction_leaf = 0.0,
                             max_features = "sqrt",
                             max_leaf_nodes = None,
                             min_impurity_decrease = 0.0,
                             bootstrap = True,
                             oob_score = False,
                             n_jobs = None,
                             verbose = 0,
                             warm_start = False,
                             class_weight = None,
                             ccp_alpha = 0.0,
                             max_samples = None)
clf.fit(train_x, train_y)

RandomForestClassifier(max_depth=100, random_state=42)

## Inference

In [36]:
preds = clf.predict(test_x)
print('Done.')

Done.


## Submission

In [37]:
submit = pd.read_csv('./sample_submission.csv')

In [38]:
submit['class'] = class_le.inverse_transform(preds)

In [41]:
submit.to_csv('./answer/submit4.csv', index=False)

In [28]:
submit

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B
